In [3]:
from selenium import webdriver
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
import random
from selenium.webdriver.support import expected_conditions as EC
import time
from random import randint
from thefuzz import process
import json
from fuzzywuzzy import process, fuzz
import requests
from datetime import datetime
import pandas as pd
from openpyxl import load_workbook
# Set the path to your downloaded Edge WebDriver executable
# Initialize the Edge WebDriver
# driver = webdriver.Edge(service=edge_service)
# driver.implicitly_wait(randint(1,3))

In [4]:
driver = webdriver.Edge()

In [5]:
secrets_file = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\user_files\secrets.json"
applied_jobs = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\log_files\applied_jobs.xlsx"
questions_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\questions.json"
search_filters_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\search_details\search_details.json"

In [6]:
def read_json(json_path):
    with open(json_path,  'r') as file:
        data = json.load(file)
    file.close()
    return data

def write_json(json_data, json_path):
    with open(json_path , 'w') as file:
        json.dump(json_data, file, indent=4)
    file.close()


def get_current_datetime():
    # Get the current date and time
    now = datetime.now()
    # Format the date and time as MM/DD/YYYY HH:MM
    formatted_datetime = now.strftime("%m/%d/%Y %H:%M")
    return formatted_datetime

def append_row_to_excel(  row_data , applied_jobs = applied_jobs):

    # Load the existing workbook
    workbook = load_workbook(applied_jobs)
    # Select the active sheet
    sheet = workbook.active

    # Append the new row
    sheet.append(row_data)

    # Save the workbook
    workbook.save(applied_jobs)
    # Close the workbook
    workbook.close()

In [7]:
secrets = read_json(secrets_file)
search_filters = read_json(search_filters_path)
qna_dict = read_json(questions_path)

salary = search_filters['Salary expectation']
workmode = search_filters['workmode']
experience = search_filters['experience level']
date_posted = search_filters['Time Posted']
companies = search_filters['company']

linkedin_url = "https://www.linkedin.com/"
password = secrets['linkedin_password']
username = secrets['linkedin_loginid']
role = search_filters['job role']
location = search_filters['Location']

In [8]:


# password = secrets['linkedin_password']
# username = secrets['linkedin_loginid']
# job_role = search_filters['job role']
# location = search_filters['Location']
# # salary = salary_dict[search_filters['Salary expectation']]
# # workmode = workmode_dict[search_filters['workmode']]
# # experience = exp_dict[search_filters['experience level']]
# # date_posted = time_dict[search_filters['Time Posted']]

In [9]:

time_dict = {
    "Any time": 0,
    "Past month": 1,
    "Past week": 2,
    "Past 24 hours": 3
}


exp_dict = {
    "Internship": 0,
    "Entry level": 1,
    "Associate": 2,
    "Mid-Senior level": 3,
    "Director": 4,
    "Executive": 5
}

salary_dict = {
    "$40,000+": 0,
    "$60,000+": 1,
    "$80,000+": 2,
    "$100,000+": 3,
    "$120,000+": 4,
    "$140,000+": 5,
    "$160,000+": 6,
    "$180,000+": 7,
    "$200,000+": 8
}

workmode_dict = {
    "On-site": 0,
    "Remote": 1,
    "Hybrid": 2
}


def check_no_matches():
    no_matches  = driver.find_elements('xpath' , ".//h1[@class='t-24 t-black t-normal text-align-center']")
    if no_matches:
        return True
    
def reset_filters():
    reset_button = driver.find_element('xpath',".//button[@aria-label='Reset applied filters']")
    reset_button.click()

def date_filter( user_date):
    date_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'timePostedRange']  " )
    date_options = date_button.find_elements( "xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    date_button.click()
    label = date_options[time_dict[user_date]].find_element("xpath" , ".//label")
    label.click()
    time.sleep(1)
    results = date_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



def exp_filter( exp_selected):
    exp_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'experience']  " )
    exp_options = exp_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    exp_button.click()
    for exp in exp_selected:
        label = exp_options[exp_dict[exp]].find_element("xpath" , ".//label")
        label.click()
    time.sleep(1)
    results = exp_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def salary_filter(  base_salary):
    salary_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'salaryBucketV2']  " )
    salary_options = salary_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    salary_button.click()
    label = salary_options[salary_dict[base_salary]].find_element("xpath" , ".//label")
    label.click()
    time.sleep(1)
    results = salary_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def workmode_filter(  workmode_selected):
    workmode_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'workplaceType']  " )
    hybrid_options = workmode_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    workmode_button.click()
    for mode in workmode_selected:
        label = hybrid_options[workmode_dict[mode]].find_element("xpath" , ".//label")
        label.click()
    time.sleep(1)
    results = workmode_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()




def company_filter(companies):

    company_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'company']  ")
    company_button.click()
    input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
    for company in companies:
    
        # input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
        input_field.clear()
        input_field.send_keys(f"{company}")
        time.sleep(0.5)
        input_field.send_keys(Keys.ARROW_DOWN)
        input_field.send_keys(Keys.ENTER)
    time.sleep(1)
    results = company_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def apply_filters():
    date_filter(date_posted)
    exp_filter(experience)
    salary_filter(salary)
    company_filter(companies)
    workmode_filter(workmode)


In [10]:
def open_linkedin_and_signin():
    driver.get(linkedin_url)
    driver.maximize_window()
    signin_page=driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

def easy_apply():
    easy_apply_button = driver.find_element( "xpath" , "//button[contains(@aria-label ,'Easy Apply filter.')]") 
    easy_apply_button.click()

#opening linkedin 
def get_url(url):
    driver.get(url)
    driver.maximize_window()
    
def sign_in(username, password):
    signin_page = driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

def job_search_and_filters():
    get_url("https://www.linkedin.com/jobs")
    role_input = driver.find_element("xpath" , ".//input[@class = 'jobs-search-box__text-input jobs-search-box__keyboard-text-input jobs-search-global-typeahead__input']")
    role_input.send_keys(role)
    time.sleep(1)
    location_input = driver.find_element("xpath" , ".//input[@aria-label = 'City, state, or zip code' and @autocomplete=  'address-level2']")
    location_input.send_keys(location)
    #submit
    role_input.send_keys(Keys.ENTER)
    time.sleep(2)
    apply_filters()
    time.sleep(2)
    easy_apply()
    

In [12]:

# Load the Q&A dictionary
def rag_query(query):
    url = "http://127.0.0.1:8000/resume_qa"
    payload = {"query": query}
    response = requests.post(url, json=payload)
    return response.json().get('response')

def qna_engine(question):
    questions_list = list(qna_dict.keys())

    question = str(question)

    if question in qna_dict:
        answer = qna_dict[question]
    else:
        threshold = 90
        matches = process.extract(question, questions_list, scorer=fuzz.WRatio, limit=3)
        filtered_matches = [match for match in matches if match[1] >= threshold]
        
        if not filtered_matches:
            answer = rag_query(question)
            qna_dict[question] = answer
        else:
            # Corrected to use the matched question string
            matched_question = filtered_matches[0][0]  # Get the actual question from the tuple
            answer = qna_dict[matched_question]
            print("THERE IS A MATCH")

    return answer
    



In [13]:
def theresubmit(driver):
    buttons = driver.find_elements( "xpath" , ".//button[@aria-label = 'Submit application']") 
    return len(buttons) and True

def get_submit(driver):
    button = driver.find_element( "xpath" , ".//button[@aria-label = 'Submit application']") 
    button.click()



def there_resume(form):
    resumes = form.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    return len(resumes) and True
    

def select_resume( number):
    resumes = driver.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    driver.execute_script("arguments[0].click();", resumes[number]) 
    resumes[number].click()   
    


def click_next():
    button = driver.find_element("xpath" , ".//button[@aria-label = 'Continue to next step' or @aria-label = 'Review your application']")
    button.click()




def attempt_click_next(current_progress, wait_time=2):
    try:
        click_next()  # Attempt to click the next button
        print("Clicked 'Next' button.")
        
        time.sleep(wait_time)  # Wait for 2 seconds to allow the progress to update
        
        # Check if progress has changed
        new_progress = get_progress_percentage(driver)
        
        if new_progress != current_progress:
            print(f"Progress changed from {current_progress}% to {new_progress}%.")
            return True  # Progress updated successfully
        else:
            print(f"Progress did not change. Still at {current_progress}%.")
            return False  # No change in progress
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False  # Handle any errors and return False

def get_progress_percentage(driver):
    # Locate the progress element and extract the value
    progress_element = driver.find_element(By.CSS_SELECTOR, "progress.artdeco-completeness-meter-linear__progress-element")
    current_progress = progress_element.get_attribute("value")
    return current_progress

def radio_select(radio_buttons):

    for radio_button in radio_buttons:


        question = radio_button.find_element(By.XPATH, ".//legend/span[@data-test-form-builder-radio-button-form-component__title]").text
        option_elements = radio_button.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = []
        
        for option in option_elements:
            options.append(  option.find_element(By.XPATH, ".//label").text)
        # print(question, options)
        answer = qna_engine(question)
        # print(answer)
        answer_match = process.extractOne(answer, options)[0] 

        # print("RADIO")
        selected_option = option_elements[options.index(answer_match)].find_element(By.XPATH, ".//input[@type='radio']")
        driver.execute_script("arguments[0].click();", selected_option)
        #driver.execute_script("arguments[0].click();", selected_option)
    

def dropdown_select(dropdowns):
    # Extract the question
    for dropdown_element in dropdowns:
        question = dropdown_element.find_element(By.XPATH, ".//label/span[@aria-hidden='true']").text
        
        # Extract the options
        options = [option.text for option in dropdown_element.find_elements(By.XPATH, ".//select/option")]
        
        # Print the question and options
        # print("Question:", question)
        # print("Options:", options)
        answer = qna_engine(question)
        # Find the best match for the answer
        answer_match = process.extractOne(answer, options)[0]
        # print(answer)
        # Select the matched option using Select
        select_element = dropdown_element.find_element(By.XPATH, ".//select[contains(@id, 'text-entity-list-form-component-formElement')]")
        select = Select(select_element)
        select.select_by_visible_text(answer_match)


def fill_single_line_text_field(text_fields):
    for text_field in text_fields:
        # Extract the question
        question = text_field.find_element(By.XPATH, ".//label").text
        
        # Print the question
        # print("Question:", question)
        answer = qna_engine(question)
        # print(answer)

        # print(" TEXT FILL")
        # Find the input field and clear any existing text
        input_field = text_field.find_element(By.XPATH, ".//input[@type='text']")
        input_field.clear()  # Clear the existing input if any
        input_field.send_keys(answer)  # Fill the answer in the input field

def fill_multiline_text_field(text_fields):
    for text_field in text_fields:
        # Extract the question
        question = text_field.find_element(By.XPATH, ".//label").text
        answer = qna_engine(question)
        # print(answer)
        # print("MULTI TEXT FILL")
        # # Print the question
        # print("Question:", question)
        
        # Find the textarea field and clear any existing text
        textarea_field = text_field.find_element(By.XPATH, ".//textarea")
        textarea_field.clear()  # Clear the existing input if any
        textarea_field.send_keys(answer)  # Fill the answer in the textarea field


def checkbox_select(fieldset_elements):
    # Extract the question
    for fieldset_element in fieldset_elements:
        question = fieldset_element.find_element(By.XPATH, ".//legend/div[@data-test-checkbox-form-title='true']").text
        answers = qna_engine(question)
        # Extract the options
        option_elements = fieldset_element.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = [option.find_element(By.XPATH, ".//label").text for option in option_elements]
        # print(answer)
        # Print the question and options
        # print("Question:", question)
        # print("Options:", options)
        if len(options) == 1:
            checkbox = option_elements[0].find_element(By.XPATH, ".//input[@type='checkbox']")
            if not checkbox.is_selected():
                checkbox.click()
                return
        # Select the matched options
        for answer in answers:
            answer_match = process.extractOne(answer, options)[0]
            option_index = options.index(answer_match)
            checkbox = option_elements[option_index].find_element(By.XPATH, ".//input[@type='checkbox']")
            if not checkbox.is_selected():
                checkbox.click()


def autofill_answer(autofill_fields):
    for autofill_field in autofill_fields:
        # Extract the question
        question = autofill_field.find_element(By.XPATH, ".//label").text
        # print("Question:", question)
        
        
        # Find the input field
        input_field = autofill_field.find_element(By.XPATH, ".//input[@aria-autocomplete='list']")
        
        # Get the answer from the user
        answer  = qna_engine(question)
        print(answer)
        # Fill the answer in the input field
        input_field.send_keys(answer)
        time.sleep(1.0)
        input_field.send_keys(Keys.DOWN)
        input_field.send_keys(Keys.ENTER)

    

In [14]:
def apply_job(job_card):

    role_name = job_card.find_element( "xpath" , ".//div[@class = 'full-width artdeco-entity-lockup__title ember-view']")
    company_name = job_card.find_element( "xpath" , ".//div[@class = 'artdeco-entity-lockup__subtitle ember-view']")
    other_data = job_card.find_elements("xpath" , ".//ul[@class = 'job-card-container__metadata-wrapper']")
    data_tuple = []
    data_tuple.append(get_current_datetime())
    data_tuple.append(role_name.text.split('\n')[0])
    data_tuple.append(company_name.text)
    for other in other_data:
        data_tuple.append(other.text)
    
    
    print("APPLYING TO " , role_name.text.split('\n')[0] , " AT " , company_name.text )
    append_row_to_excel(data_tuple)
    # append_row_to_excel(data_tuple)




    job_card.click()
    try:
        easy_apply_button = driver.find_element("xpath" , ".//button[contains(@aria-label , 'Easy Apply to')]")
        easy_apply_button.click()
    except Exception as e:
        return
    print("clicked easy apply")
    
    filledresume = False
    applying = True

    while applying:
        
        current_progress = get_progress_percentage(driver)
        print("current progress is",current_progress)

        if theresubmit(driver):
            print("found submit")
            get_submit(driver)
            break

        
        apply = driver.find_element("xpath" , ".//div[@class='jobs-easy-apply-content']")
        form = apply.find_element("xpath", ".//form")

        # Handle resume selection
        if not filledresume and there_resume(form):
            try:
                select_resume(0)
                filledresume = True
            except Exception as e:
                print(f"Error selecting resume: {e}")

        # Handle radio buttons selection
        form = driver.find_element("xpath", ".//form")
        radios = form.find_elements("xpath", ".//fieldset[@data-test-form-builder-radio-button-form-component = 'true']")
        if radios:
            try:
                radio_select(radios)
            except Exception as e:
                print(f"Error selecting radio buttons: {e}")

        # Handle dropdown selection
        form = driver.find_element("xpath", ".//form")
        text_dropdowns = form.find_elements("xpath", ".//div[@data-test-text-entity-list-form-component='']")
        if text_dropdowns:
            try:
                dropdown_select(text_dropdowns)
            except Exception as e:
                print(f"Error selecting dropdowns: {e}")

        # Handle single-line text fields
        form = driver.find_element("xpath", ".//form")
        singleline_textfields = form.find_elements("xpath", ".//div[@data-test-single-line-text-form-component='']")
        if singleline_textfields:
            try:
                fill_single_line_text_field(singleline_textfields)
            except Exception as e:
                print(f"Error filling single-line text fields: {e}")

        # Handle multiline text fields
        form = driver.find_element("xpath", ".//form")
        multiline_fields = form.find_elements("xpath", ".//div[@data-test-multiline-text-form-component='']")
        if multiline_fields:
            try:
                fill_multiline_text_field(multiline_fields)
            except Exception as e:
                print(f"Error filling multiline text fields: {e}")

        # Handle checkboxes
        form = driver.find_element("xpath", ".//form")
        checkboxes = form.find_elements("xpath", ".//fieldset[@data-test-checkbox-form-component='true']")
        if checkboxes:
            try:
                checkbox_select(checkboxes)
            except Exception as e:
                print(f"Error selecting checkboxes: {e}")

        # Handle autofill fields
        form = driver.find_element("xpath", ".//form")
        autofills = form.find_elements("xpath", ".//div[@data-test-single-typeahead-entity-form-component='']")
        # print(f"NUMBER OF AUTOFILLS: {len(autofills)}")
        if autofills:
            try:
                autofill_answer(autofills)
            except Exception as e:
                print(f"Error filling autofill fields: {e}")


        next_buttons =  driver.find_element("xpath" , ".//button[@aria-label = 'Continue to next step' or @aria-label = 'Review your application']")
        if next_buttons:
            attempt = False
            
            while not attempt:
                print("attempting to click")
                next_buttons =  driver.find_element("xpath" , ".//button[@aria-label = 'Continue to next step' or @aria-label = 'Review your application']")
                attempt = attempt_click_next(current_progress)
                if not attempt:
                    print("Manually fill this page.")
                    time.sleep(20)  # Wait for 20 seconds before the next attempt

    time.sleep(2)
    closingpopup = True
    while closingpopup: 
        
        print("finding exit")
        exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
        if exit_buttons:
            exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
            exit_buttons[0].click()
            print("exited")
            print("APPLIED TO " , role_name.text.split('\n')[0] , " AT " , company_name.text )
            append_row_to_excel(data_tuple)
            return
        


In [15]:
get_current_datetime()

'10/04/2024 07:18'

In [16]:
def iterator( driver):
    currentpage = 1
    nextpageexists = True
    
    counter = 0
    while nextpageexists:
        
        search_results = driver.find_element("xpath" , ".//div[contains(@class, 'jobs-search-results-list')]")
        list_of_jobs = search_results.find_elements( "xpath" , "//li[@data-occludable-job-id]")
        
        attempt_job = 0
        while attempt_job < 3:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", list_of_jobs[-1])
                
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
       
        for job in list_of_jobs:
            
            start_time = time.time()
            time.sleep(1.0)
            job.click() 
            time.sleep(0.2)
            apply_job(job)
            counter+=1
            print(counter, " jobs")
            write_json(qna_dict, questions_path)
            end_time = time.time()

            print("time taken" , end_time - start_time)
            
        
        

        
        attempt_job = 0
        while attempt_job < 3:
            try:
                
                driver.execute_script("arguments[0].click();", list_of_jobs[-1])
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
        
        # next_pages = driver.find_elements("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']")
        currentpage+=1


        attempts = 0
        while attempts < 5:
            try:
                driver.find_element("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']").click()
                nextpageexists = True
                attempts = 5
            except Exception as e:
                attempts += 1
                nextpageexists = False

        if nextpageexists:
            print("moving to next page")
            
            
        else:
            print("ending function")
            return
    return

In [17]:
open_linkedin_and_signin()
job_search_and_filters()
time.sleep(3)
iterator(driver)

APPLYING TO  Data Scientist  AT  Diamondpick
clicked easy apply
current progress is 0
THERE IS A MATCH
attempting to click
Clicked 'Next' button.
Progress changed from 0% to 50%.
current progress is 50
attempting to click
Clicked 'Next' button.
Progress changed from 50% to 100%.
current progress is 100
found submit
finding exit


ElementNotInteractableException: Message: element not interactable
  (Session info: MicrosoftEdge=129.0.2792.65)
Stacktrace:
	GetHandleVerifier [0x00007FF6551CCEC5+12997]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF65544BCA4+1897908]
	(No symbol) [0x00007FF654F7F005]
	(No symbol) [0x00007FF654FC79ED]
	(No symbol) [0x00007FF654FBC65A]
	(No symbol) [0x00007FF654FE7F4A]
	(No symbol) [0x00007FF654FBC09D]
	(No symbol) [0x00007FF654FBBF6D]
	(No symbol) [0x00007FF654FE8230]
	(No symbol) [0x00007FF654FBC09D]
	(No symbol) [0x00007FF655002887]
	(No symbol) [0x00007FF654FE7BB3]
	(No symbol) [0x00007FF654FBB5AC]
	(No symbol) [0x00007FF654FBAA8D]
	(No symbol) [0x00007FF654FBB171]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF655379DD4+1038052]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6550F08BF+56655]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6550E3063+1267]
	Microsoft::Applications::Events::Even